In [ ]:
import numpy as np
import pandas as pd
import nltk
import gensim
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from gensim.models import Word2Vec,KeyedVectors

# Load Google's pre-trained Word2Vec model.
# model = Word2Vec.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Embedings/GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


 **5 Examples Word Similarity**

In [ ]:
model.similarity('day','Monday')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.4604319

In [ ]:
model.similarity('Tuesday','Monday')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9720569

In [ ]:
model.similarity('women','men')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7674937

In [ ]:
model.similarity('queen','king')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.6510957

In [ ]:
model.similarity('girl','boy')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.8543272

5 examples of Word Analogies

In [ ]:
print('Analogies : {} , {}'.format(model.similarity('comb','hair'),model.similarity('hammer','nail')))

Analogies : 0.3850337266921997 , 0.42197614908218384


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print('Analogies : {} , {}'.format(model.similarity('white','black'),model.similarity('up','down')))

Analogies : 0.8092214465141296 , 0.6396992206573486


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print('Analogies : {} , {}'.format(model.similarity('short','light'),model.similarity('long','heavy')))

Analogies : 0.1567705124616623 , 0.24463792145252228


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print('Analogies : {} , {}'.format(model.similarity('like','love'),model.similarity('dislike','hate')))

Analogies : 0.3671387732028961 , 0.6013234853744507


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Tweets.csv')

In [ ]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
data.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [ ]:
Useful_Columns = ['airline_sentiment','text']

In [ ]:
data = data[Useful_Columns]

In [ ]:
import re
data['text'] = data['text'].map(lambda x:re.sub('@\w*','',str(x)))
data['text'] = data['text'].map(lambda x:re.sub('[^a-zA-Z]',' ',str(x)))
data['text'] = data['text'].map(lambda x:re.sub('http.*','',str(x)))

In [ ]:
data['text'] = data['text'].map(lambda x:str(x).lower())

In [ ]:
data['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [ ]:
data['airline_sentiment'] = data['airline_sentiment'].map({'positive':1,'neutral':0,'negative':-1})

In [ ]:
data.head()

,airline_sentiment,text
0,0,what said
1,1,plus you ve added commercials to the experien...
2,0,i didn t today must mean i need to take an...
3,-1,it s really aggressive to blast obnoxious en...
4,-1,and it s a really big bad thing about it


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
X = []
for text in data['text']:
  X.append([word for word in text.strip().split() if not word in set(stopwords.words('english'))])

In [ ]:
Y = data['airline_sentiment']

In [ ]:
X[:4]

[['said'],
 ['plus', 'added', 'commercials', 'experience', 'tacky'],
 ['today', 'must', 'mean', 'need', 'take', 'another', 'trip'],
 ['really',
  'aggressive',
  'blast',
  'obnoxious',
  'entertainment',
  'guests',
  'faces',
  'amp',
  'little',
  'recourse']]

In [ ]:
X_final = []
for x in X:
  d = np.zeros(300)
  for y in x:
    try:
      d = d + model[y]
    except:
      pass
  X_final.append(d)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_final,Y,test_size=0.2,random_state=42)

In [ ]:
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
acc_score = accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred),'\n',cm,'\n',acc_score)

              precision    recall  f1-score   support

          -1       0.84      0.88      0.86      1889
           0       0.58      0.54      0.56       580
           1       0.74      0.68      0.71       459

    accuracy                           0.78      2928
   macro avg       0.72      0.70      0.71      2928
weighted avg       0.78      0.78      0.78      2928
 
 [[1663  162   64]
 [ 221  314   45]
 [  85   62  312]] 
 0.7817622950819673


Training Word Embedings

In [ ]:
model1 = Word2Vec(X, min_count=1)

In [ ]:
print(model1)

Word2Vec(vocab=10972, size=100, alpha=0.025)


In [ ]:
X_final = []
for x in X:
  d = np.zeros(300)
  for y in x:
    try:
      d = d + model1[y]
    except:
      pass
  X_final.append(d)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_final,Y,test_size=0.2,random_state=42)

In [ ]:
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
acc_score = accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred),'\n',cm,'\n',acc_score)

              precision    recall  f1-score   support

          -1       0.65      1.00      0.78      1889
           0       0.00      0.00      0.00       580
           1       0.00      0.00      0.00       459

    accuracy                           0.65      2928
   macro avg       0.22      0.33      0.26      2928
weighted avg       0.42      0.65      0.51      2928
 
 [[1889    0    0]
 [ 580    0    0]
 [ 459    0    0]] 
 0.6451502732240437


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
